In [ ]:
#
# BSD 3-Clause License
#
# This file is part of the Basalt project.
# https://gitlab.com/VladyslavUsenko/basalt.git
#
# Copyright (c) 2019-2021, Vladyslav Usenko and Nikolaus Demmel.
# All rights reserved.
#

In [1]:
from pathlib import Path
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['text.usetex'] = True
from mpl_toolkits.axes_grid1 import make_axes_locatable

from pnec.simulation.load_experiment import load_problem
from pnec.common import pnec_energy_rotations
from pnec.math import rotation_between_points


In [2]:
def mesh_of_rotations(n: int, m: int, max_pitch: float, max_yaw: float, base_rotation: np.array):
    # returns a mesh of size (n, m) of rotations around the base_rotation with a maximum pitch and yaw
    pitch = np.linspace(-max_pitch, max_pitch, n)
    yaws = np.linspace(-max_yaw, max_yaw, m)

    pitch_v, yaw_v = np.meshgrid(pitch, yaws, indexing='ij')

    euler_angles = np.stack(
        [yaw_v.ravel(), pitch_v.ravel()], axis=1)

    scipy_rotations = R.from_euler(
        'xy', euler_angles, degrees=True) * R.from_matrix(base_rotation)
    return scipy_rotations.as_matrix().reshape((n, m, 3, 3)), pitch_v, yaw_v

In [16]:
def evaluate_experiment(path: Path, omnidirectional: bool, problem_num: int, use_prediction: bool, pred_path: Path):
    n = 1 + 100
    m = 1 + 100
    max_pitch = 5
    max_yaw = 5
    gt_pose, bvs_1, bvs_2, sigmas, pred_pose = load_problem(path, omnidirectional, problem_num, True, pred_path)
    
    if use_prediction:
        rots, ppitch, yyaw = mesh_of_rotations(n, m, max_pitch, max_yaw, gt_pose.rotationMatrix())
    else:
        rots, ppitch, yyaw = mesh_of_rotations(n, m, max_pitch, max_yaw, pred_pose.rotationMatrix())

    # use two translations, 1. t1 = t_pred, 2. angle(t1, t2) = theta
    theta = 30 * np.pi/180
    phi = 10 * np.pi/180
    t1 = pred_pose.translation()
    rotational_offset = rotation_between_points(np.array([0.0, 0.0, 1.0]), t1)
    t2 = np.matmul(rotational_offset, np.array([np.sin(phi)*np.sin(theta), np.cos(phi)
                                      * np.sin(theta), np.cos(theta)]))

    energy_1 = pnec_energy_rotations(rots, t1, bvs_1, bvs_2, sigmas)
    energy_2 = pnec_energy_rotations(rots, t2, bvs_1, bvs_2, sigmas)

    return {'energy_1': energy_1, 'energy_2': energy_2, 'ppitch': ppitch, 'yyaw': yyaw, 'rotations': rots}


### Load Problem

In [17]:
base_folder = Path('/Volumes/Samsung_T5/new_experiments/normal')
omnidirectional = True
noise_type = 'anisotropic_inhomogeneous'
noise_level = '1.000000'
problem_number = 13

if omnidirectional:
        camera_folder = base_folder.joinpath('omnidirectional')
else:
    camera_folder = base_folder.joinpath('pinhole')
path_1 = camera_folder.joinpath(noise_type, noise_level)
path_2 = camera_folder.joinpath(noise_type + '_no_t', noise_level)

use_pred = True

results_1 = evaluate_experiment(
    path_1, omnidirectional, problem_number, use_pred, path_1.joinpath('alt_timing1', 'pnec_solution.csv'))

results_2 = evaluate_experiment(
    path_2, omnidirectional, problem_number, use_pred, path_2.joinpath('alt_timing1', 'pnec_solution.csv'))


### Plotting for two different translations t1, t2

In [192]:
plt.close("all")

hfac = 2.1*1.61803398875

hsize = 7
fig, axs = plt.subplots(2, 2, figsize=(
    hsize, 1.2 * hsize/hfac), sharex=True, sharey=True)
linewidth = 1.0
color = 'r'
level = 5
mark_minimum = False

for i, results in enumerate([results_1, results_2]):
    vmin = min(results['energy_1'].min(), results['energy_2'].min())
    vmax = max(results['energy_1'].max(), results['energy_2'].max())
    for j, energy in enumerate([results['energy_1'], results['energy_2']]):
        ax = axs[i, j]
        cm = ax.pcolor(results['ppitch'], results['yyaw'], energy,
                       shading='auto', vmin=vmin, vmax=vmax)
        conm = ax.contour(results['ppitch'], results['yyaw'], energy, levels=level,
                  colors=color, linewidths=linewidth)

        if j == 1:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.05)
            cbar = fig.colorbar(cm, cax=cax, orientation='vertical')
            cbar.formatter.set_powerlimits((0, 0))

        if mark_minimum:
            size = 10.0
            r_argmin = np.unravel_index(
                energy.argmin(), energy.shape)
            ax.scatter(results['ppitch'][r_argmin],
                       results['yyaw'][r_argmin], c='r', s=size)


ax = axs[0, 0]
ax.set_ylabel("yaw to\n$R_{\\textnormal{pred}}$ [deg]", labelpad=1)
ax.set_title(r"Experiment {\scshape w/o t}: \quad $t = t_{\textnormal{pred}}$")
ax = axs[0, 1]
ax.set_title(
    r"{\scshape w/o t}: \quad $\angle (t, t_{\textnormal{pred}}) = 30^{\circ}$")
ax = axs[1, 0]
ax.set_ylabel("yaw to\n$R_{\\textnormal{pred}}$ [deg]", labelpad=1)
ax.set_title(
    r"Experiment {\scshape w/\phantom{o} t}: \quad $t = t_{\textnormal{pred}}$")
ax.set_xlabel(r"pitch to $R_{\textnormal{pred}}$ [deg]", labelpad=-1)
ax = axs[1, 1]
ax.set_title(
    r"{\scshape w/\phantom{o} t}: \quad $\angle (t, t_{\textnormal{pred}}) = 30^{\circ}$")
ax.set_xlabel(r"pitch to $R_{\textnormal{pred}}$ [deg]", labelpad=-1)

plt.tight_layout()
plt.show()
plt.savefig("energy_test.pdf", bbox_inches='tight')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …